In [14]:
import routeros_api2
import time
import sys
import random
import matplotlib
matplotlib.use('Qt5Agg')
from PyQt5 import QtCore, QtWidgets
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar
from matplotlib.figure import Figure
from collections import deque
from statistics import mean

In [15]:
main_interface=routeros_api2.Api(address='192.168.88.4')

In [3]:
#azimuth=[27, 19, 12,4,-4,-12,-19,-27,27, 19, 12,4,176,168,161,153,27, 19, 12,4,176,168,161,153,27, 19, 12,4,176,168,161,153,27, 19, 12,4,176,168,161,153,27, 19, 12,4,176,168,161,153,27, 19, 12,4,176,168,161,153,27, 19, 12,4,176,168,161,153]
zenith=[-27, -27, -27, -27, -27, -27, -27, -27, -19, -19, -19, -19, -19, -19, -19, -19, -12, -12, -12, -12, -12, -12, -12, -12, -4, -4, -4, -4, -4, -4, -4, -4, 4, 4, 4, 4, 4, 4, 4, 4, 12, 12, 12, 12, 12, 12, 12, 12, 19, 19, 19, 19, 19, 19, 19, 19, 27, 27, 27, 27, 27, 27, 27, 27]

In [4]:
azimuth=[27, 19, 12,4,-4,-12,-19,-27]*8


In [20]:
def align():
    routeros_api2.Api.talk(main_interface,'/interface/w60g/align\n=numbers="wlan60-2"\n=once')
    time.sleep(10)
    recv=routeros_api2.Api.talk(main_interface,'/interface/w60g/monitor\n=numbers="wlan60-2"\n=once')
    print(recv)
    #return recv[0]

In [6]:
def monitor():
    recv= routeros_api2.Api.talk(main_interface,'/interface/w60g/monitor\n=numbers="wlan60-2"\n=once')
    print(recv)

In [7]:
def beamset(i):
    recv=routeros_api2.Api.talk(main_interface,'/interface/w60g/set\n=numbers="wlan60-2"\n=tx-sector='+str(i))
    #print(recv)

In [16]:
def monitor_send():
    read=routeros_api2.Api.send(main_interface,'/interface/w60g/monitor\n=numbers="wlan60-2"\n=interval=.02')
    print(read)

In [35]:
def align_send():
    read=routeros_api2.Api.send(main_interface,'/interface/w60g/align\n=numbers="wlan60-2"\n=once')
    print(read)

In [18]:
monitor_send()

KeyboardInterrupt: 

In [ ]:
txsector=0
mcs=0
per=0
rss=-60
avgrss=deque(maxlen=1)
count=0
def rssi():
    data=[]
    global txsector
    global mcs
    global per
    global rss
    global avgrss
    
    #time.sleep(.1)
    
    
    recv=(routeros_api2.Api.talk(main_interface,'/interface/w60g/monitor\n=numbers="wlan60-2"\n=once'))
    #print(recv[0])
    if(recv[0]['connected']=='true'):
        rss=recv[0]['rssi']
        #print(rss)
        per=recv[0]['tx-packet-error-rate']
        txsector=recv[0]['tx-sector']
        mcs=recv[0]['tx-mcs']
        avgrss.append(int(rss))
        if len(avgrss)==1:
            rss=mean(avgrss)
#         if (len(avgrss)==3 and mean(avgrss)<-58):
#             beamset(20)
#         else:
#             beamset(59)
    else:
        rss=-70
        mcs=0
        per=100
    monitor()
    time.sleep(.2)
    return rss

In [ ]:
import dash
from dash.dependencies import Output, Input
from dash import dcc
from dash import html
import plotly
import random
import plotly.graph_objs as go
%tb
qlen=5
angles=[27, 19, 12,4,176,168,161,153,27, 19, 12,4,176,168,161,153,27, 19, 12,4,176,168,161,153,27, 19, 12,4,176,168,161,153,27, 19, 12,4,176,168,161,153,27, 19, 12,4,176,168,161,153,27, 19, 12,4,176,168,161,153]
X = deque(maxlen = qlen)
X.append(1)

Y = deque(maxlen = qlen)
Y.append(-65)

X1 = deque(maxlen = qlen)
X1.append(1)

Y1 = deque(maxlen = qlen)
Y1.append(0)


X2 = deque(maxlen = qlen)
X2.append(1)

Y2 = deque(maxlen = qlen)
Y2.append(0)

X3 = deque(maxlen = qlen)
X3.append(1)

Y3 = deque(maxlen = qlen)
Y3.append(0)

X4 = deque(maxlen = qlen)
X4.append(1)

Y4 = deque(maxlen = qlen)
Y4.append(0)

app = dash.Dash(__name__)


GRAPH_INTERVAL = 400

app = dash.Dash(
    __name__,
    meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1"}],
)
#app.title = "Wind Speed Dashboard"

server = app.server

app_color = {"graph_bg": "#082255", "graph_line": "#007ACE"}

app.layout = html.Div(
    [

        html.Div(
            [
                # wind speed
                html.Div(
                    [
                        html.Div(
                            [html.H6("60 GHz Receiver", className="graph__title")]
                        ),
                        dcc.Graph(
                            id="wind-speed1",
                            figure=dict(
                                layout=dict(
                                    plot_bgcolor=app_color["graph_bg"],
                                    paper_bgcolor=app_color["graph_bg"],
                                )
                            ),
                        ),
                        dcc.Interval(
                            id="wind-speed-update1",
                            interval=int(GRAPH_INTERVAL),
                            n_intervals=0,
                        ),
                            html.Div(
                        [
                                html.Div(
                                    [
                                html.Div(
                                    [
                                        html.Div(
                                    #[html.H6("60 GHz Receiver", className="graph__title")]
                                ),
                                dcc.Graph(
                                    id="wind-speed2",
                                    figure=dict(
                                        layout=dict(
                                            plot_bgcolor=app_color["graph_bg"],
                                            paper_bgcolor=app_color["graph_bg"],
                                        )
                                    ),
                                ),
                                dcc.Interval(
                                    id="wind-speed-update2",
                                    interval=int(GRAPH_INTERVAL),
                                    n_intervals=0,
                                ),
                                    ],
                                    className="one-half column",
                                ),
                                html.Div(
                                    [
                                        html.Div(
                                    #[html.H6("60 GHz Receiver", className="graph__title")]
                                ),
                                dcc.Graph(
                                    id="wind-speed5",
                                    figure=dict(
                                        layout=dict(
                                            plot_bgcolor=app_color["graph_bg"],
                                            paper_bgcolor=app_color["graph_bg"],
                                        )
                                    ),
                                ),
                                dcc.Interval(
                                    id="wind-speed-update5",
                                    interval=int(GRAPH_INTERVAL),
                                    n_intervals=0,
                                ),
                                    ], 
                                    className="one-half column",
                                ),
                                
                             ], 
                             #className="two-thirds column wind__speed__container",
                         ),# wind direction

                        ],
                        #className="two-thirds column wind__speed__container",
                    ),
                    ],
                    className="two-thirds column wind__speed__container",
                ),
                
                html.Div(
                    [
                        html.Div(
                           [html.H6("60 GHz Receiver", className="graph__title")]
                        ),
                        dcc.Graph(
                            id="wind-speed3",
                            figure=dict(
                                layout=dict(
                                    plot_bgcolor=app_color["graph_bg"],
                                    paper_bgcolor=app_color["graph_bg"],
                                )
                            ),
                        ),
                        dcc.Interval(
                            id="wind-speed-update3",
                            interval=int(GRAPH_INTERVAL),
                            n_intervals=0,
                        ),
                            html.Div(
                        [
                            # histogram
                            html.Div(
                                [
                                    html.Div(
                              [html.H6("", className="graph__title")]
                            ),
                            dcc.Graph(
                                id="wind-speed4",
                                figure=dict(
                                    layout=dict(
                                        plot_bgcolor=app_color["graph_bg"],
                                        paper_bgcolor=app_color["graph_bg"],
                                    )
                                ),
                            ),
                            dcc.Interval(
                                id="wind-speed-update4",
                                interval=int(GRAPH_INTERVAL),
                                n_intervals=0,
                            ),
                                ],
                                className="graph__container first",
                            ),
                            # wind direction

                        ],
                        #className="two-thirds column wind__speed__container",
                    ),
                    ],
                    className="one-third column wind__speed__container",
                ),
                
            ],
            className="app__content",
        ),
    ],
    className="app__container",
)


@app.callback(
    Output('wind-speed1', 'figure'),
    [ Input('wind-speed-update1', 'n_intervals') ]
)

def update_graph_scatter(n):
    X.append(X[-1]+.3)
    if len(Y)==qlen:
        del Y[0]
    Y.append(rssi())

    data = plotly.graph_objs.Scatter(
            x=list(X),
            y=list(Y),
            line=dict(color="red", width=5),
            name='Scatter',
            mode= 'lines+markers'
    )
    
    layout = dict(
    #plot_bgcolor=app_color["graph_bg"],
    #paper_bgcolor=app_color["graph_bg"],
    font=dict(
        family="Courier New, monospace",
        size=25,
        color="#000000"
    ),
    #height=500,
    #width =700,
    #="Time (S)",
    title="<b> Received Signal Strength <b>",
    xaxis={
        "range": [min(X),max(X)],
        "showline": True,
        "zeroline": False,
        "fixedrange": True,
        "title": "Time (S)",
    },
    yaxis={
        "range": [min(Y),max(Y)],
        "showgrid": True,
        "showline": True,
        "fixedrange": True,
        "zeroline": False,
       # "gridcolor": app_color["graph_line"],
        "nticks": 10,
        "title": 'RSS (dBm)'
    },)

    return {'data': [data],
            'layout' : layout}

@app.callback(
    Output('wind-speed2', 'figure'),
    [ Input('wind-speed-update2', 'n_intervals') ]
)

def update_graph_scatter(n):
    X1.append(X1[-1]+.3)
    if len(Y1)==qlen:
        del Y1[0]
    Y1.append(per)

    data = plotly.graph_objs.Scatter(
            x=list(X1),
            y=list(Y1),
            line=dict(color="green", width=5),
            name='Scatter',
            mode= 'lines+markers'
    )

    return {'data': [data],
            'layout' : go.Layout(title=" <b> Packet Error Rate <b>",xaxis=dict(range=[min(X1),max(X1)]),yaxis = dict(range = [0,105],nticks=12), xaxis_title="Time (S)",
    yaxis_title="%", font=dict(
        family="Courier New, monospace",
        size=25,
        color="#000000"
    ))}


@app.callback(
    Output('wind-speed5', 'figure'),
    [ Input('wind-speed-update5', 'n_intervals') ]
)

def update_graph_scatter(n):
    X2.append(X2[-1]+.3)
    if len(Y2)==qlen:
        del Y2[0]
    Y2.append(mcs)

    data = plotly.graph_objs.Scatter(
            x=list(X2),
            y=list(Y2),
            line=dict(color="blue", width=5),
            name='Scatter',
            mode= 'lines+markers'
    )

    return {'data': [data],
            'layout' : go.Layout(title="<b> Modulation and Coding Scheme <b>",xaxis=dict(range=[min(X1),max(X1)]),yaxis = dict(range = [0,9],nticks=9), xaxis_title="Time (S)",
    yaxis_title="Index", font=dict(
        family="Courier New, monospace",
        size=20,
        color="#000000"
    ))}





@app.callback(
    Output('wind-speed3', 'figure'),
    [ Input('wind-speed-update3', 'n_intervals') ]
)

def update_graph_scatter(n):
    X3.append(X3[-1]+1)
    if len(Y3)==19:
        del Y3[0]
    Y3.append(txsector)

#     data = plotly.graph_objs.Scatter(
#             x=list(X4),
#             y=list(Y4),

#             name='Scatter',
#             mode= 'lines+markers'
#     )
    

    data = (go.Barpolar(
        r=[4*(int(90)/100.0)],
        theta=[azimuth[int(txsector)]],
        width=[20],
       # marker_color=["#E4FF87", '#709BFF', '#709BFF', '#FFAA70', '#FFAA70', '#FFDF70', '#B6FFB4'],
        marker_color='#B6FFB4',
        marker_line_color="black",
        marker_line_width=2,
        opacity=0.8,
       
    ))

    layout = go.Layout(autosize=False,
    width=500,
    height=500,
    font=dict(
        family="Courier New, monospace",
        size=20,
        color="#000000"
    ),
    #height=500,
    #width =700,
    #="Time (S)",
    title="<b> Azimuth Angle <b>",  
    polar=dict(
    radialaxis_tickfont_size = 12,
      angularaxis = dict(
        tickfont_size=20,
        rotation=0, # start position of angular axis
        direction="counterclockwise"
      )),
    margin=dict(
        l=50,
        r=53,
        b=30,
        t=70,
        pad=4
    ))
    
    return {'data': [data],'layout':layout}






@app.callback(
    Output('wind-speed4', 'figure'),
    [ Input('wind-speed-update4', 'n_intervals') ]
)





# fig.update_layout(
#     template=None,
# #     polar = dict(
# #         radialaxis = dict(range=[0, 5], showticklabels=False, ticks=''),
# #         angularaxis = dict(showticklabels=False, ticks='')
# #     )
# # )







def update_graph_scatter(n):
    X4.append(X4[-1]+1)
    if len(Y4)==19:
        del Y4[0]
    Y4.append(txsector)

#     data = plotly.graph_objs.Scatter(
#             x=list(X4),
#             y=list(Y4),

#             name='Scatter',
#             mode= 'lines+markers'
#     )
    

    data = (go.Barpolar(
        r=[4*(int(90)/100.0)],
        theta=[zenith[int(txsector)]],
        width=[20],
       # marker_color=["#E4FF87", '#709BFF', '#709BFF', '#FFAA70', '#FFAA70', '#FFDF70', '#B6FFB4'],
        marker_color='#FFAA70',
        marker_line_color="black",
        marker_line_width=2,
        opacity=0.8,
       
    ))

    layout = go.Layout(autosize=True,
    width=500,
    height=400,
     font=dict(
        family="Courier New, monospace",
        size=20,
        color="#000000"
    ),
    #height=500,
    #width =700,
    #="Time (S)",
    title="<b> Zenith Angle <b>",       
    polar = dict(
    sector = [-90,90],
      radialaxis_tickfont_size = 12,
      angularaxis = dict(
        tickfont_size=23,
        rotation=0, # start position of angular axis
        direction="counterclockwise"
      )
    ),
    margin=dict(
        l=30,
        r=33,
        b=10,
        t=70,
        pad=4
    ))
    
    return {'data': [data],'layout':layout}


if __name__ == '__main__':
    app.run_server()
